### 1.导入相关的库

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder # 对目标标签进行编码，其值介于0和n_classes-1之间。该转换器应用于编码目标值，即y，而不是输入X。
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [3]:
# 加载数据集
train = pd.read_csv('/home/mw/input/minsu2472/train_data.csv')
test = pd.read_csv('/home/mw/input/minsu2472/test_data.csv')

In [4]:
#数据探索
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59288 entries, 0 to 59287
Data columns (total 27 columns):
Unnamed: 0     59288 non-null int64
数据ID           59288 non-null object
容纳人数           59288 non-null int64
便利设施           59288 non-null object
洗手间数量          59128 non-null float64
床的数量           59184 non-null float64
床的类型           59288 non-null int64
卧室数量           59210 non-null float64
取消条款           59288 non-null int64
所在城市           59288 non-null int64
清洁费            59288 non-null int64
首次评论日期         46586 non-null object
房主是否有个人资料图片    59128 non-null object
房主身份是否验证       59128 non-null object
房主回复率          44647 non-null object
何时成为房主         59128 non-null object
是否支持随即预订       59288 non-null int64
最近评论日期         46616 non-null object
维度             59288 non-null float64
经度             59288 non-null float64
民宿周边           53779 non-null object
评论个数           59288 non-null int64
房产类型           59288 non-null int64
民宿评分           45893 non-null float64
房型    

In [5]:
train.head()

,Unnamed: 0,数据ID,容纳人数,便利设施,洗手间数量,床的数量,床的类型,卧室数量,取消条款,所在城市,清洁费,首次评论日期,房主是否有个人资料图片,房主身份是否验证,房主回复率,何时成为房主,是否支持随即预订,最近评论日期,维度,经度,民宿周边,评论个数,房产类型,民宿评分,房型,邮编,价格
0,0,train_0,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.5,3.0,4,2.0,0,3,0,2015-05-07,t,t,NaN,2015-02-25,0,2016-06-26,34.109039,-118.273390,Los Feliz,12,17,97.0,0,90027,64.918531
1,1,train_1,2,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",1.0,1.0,4,1.0,2,4,1,2016-07-02,t,t,NaN,2009-10-27,1,2016-07-31,40.812897,-73.919163,Mott Haven,6,0,87.0,0,10454,54.918531
2,2,train_2,4,"{TV,""Air conditioning"",Kitchen,Heating,""Smoke ...",1.0,2.0,4,0.0,2,4,1,2017-07-01,t,f,100%,2017-06-29,1,2017-07-31,40.737643,-73.953309,Greenpoint,4,0,80.0,0,11222,73.219281
3,3,train_3,2,{},1.0,1.0,4,1.0,0,5,1,NaN,t,t,NaN,2013-03-19,0,NaN,37.759935,-122.420558,Mission District,0,0,NaN,1,94110,64.093909
4,4,train_4,3,"{Internet,""Wireless Internet"",""Air conditionin...",1.0,1.0,4,1.0,1,4,1,2014-04-30,t,t,100%,2011-07-30,0,2016-05-22,40.683363,-73.949490,Bedford-Stuyvesant,16,0,99.0,0,11216,68.454901


### 2.数据特征预处理

In [6]:
#数据特征预处理
# 预处理1 便利设施的数量
train['便利设施_num'] = train['便利设施'].str.split(',')
int_equipments = []
for item in train['便利设施_num']:
    num = len(item)
    int_equipments.append(num)
train['便利设施_num'] = int_equipments
test['便利设施_num'] = test['便利设施'].str.split(',')
int_equipments_test = []
for item in test['便利设施_num']:
    num = len(item)
    int_equipments_test.append(num)
test['便利设施_num'] = int_equipments_test


In [7]:
# 预处理2
for keyword in ['Internet','Kitchen','Air','Pets','Washer','aid','Dryer','Essentials','Hangers','TV','Heating','parking','24-hour']:
    train[keyword] = train['便利设施'].str.contains(keyword).astype('str')
    test[keyword] = test['便利设施'].str.contains(keyword).astype('str')
test.head()

,Unnamed: 0,数据ID,容纳人数,便利设施,洗手间数量,床的数量,床的类型,卧室数量,取消条款,所在城市,清洁费,首次评论日期,房主是否有个人资料图片,房主身份是否验证,房主回复率,何时成为房主,是否支持随即预订,最近评论日期,维度,经度,民宿周边,评论个数,房产类型,民宿评分,房型,邮编,便利设施_num,Internet,Kitchen,Air,Pets,Washer,aid,Dryer,Essentials,Hangers,TV,Heating,parking,24-hour
0,0,test_0,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.5,1.0,4,1.0,2,1,1,2015-05-25,t,t,100%,2015-05-20,1,2017-01-01,41.849684,-87.676270,Pilsen,17,17,97.0,1,60608,19,True,True,True,False,True,True,True,True,False,True,True,False,False
1,1,test_1,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",2.0,1.0,4,1.0,2,3,1,2015-11-09,t,t,100%,2015-09-08,0,2015-11-15,34.068613,-118.246455,Echo Park,2,0,100.0,0,90012,12,True,True,True,False,True,False,True,True,False,True,True,False,False
2,2,test_2,5,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",1.0,3.0,4,2.0,1,4,1,2017-05-15,t,t,100%,2017-05-06,1,2017-09-25,40.701958,-73.917352,Bushwick,25,0,88.0,0,11237,25,True,True,True,True,False,True,False,True,True,True,True,False,False
3,3,test_3,6,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",1.0,3.0,4,1.0,2,4,1,2012-11-12,t,t,70%,2009-02-06,0,2017-07-29,40.742959,-73.990820,Flatiron District,12,0,82.0,0,10010,9,True,True,True,False,False,False,False,False,False,True,True,False,False
4,4,test_4,2,"{Internet,""Wireless Internet"",""Air conditionin...",1.0,1.0,4,1.0,0,3,1,2017-02-17,t,t,100%,2015-10-20,0,2017-03-25,34.046473,-117.734095,NaN,2,17,100.0,1,91766,17,True,False,True,False,False,False,False,True,True,False,True,True,False


In [8]:
# 预处理3
train['民宿周边'] = train['民宿周边'].fillna(-999)
test['民宿周边'] = test['民宿周边'].fillna(-999)
surround = []
for v in train['民宿周边']:
    if v == -999:
        new_v = 0
    else:
        new_v = 1
    surround.append(new_v)
train['民宿周边'] = surround
surround_test= []
for v in test['民宿周边']:
    if v == -999:
        new_v = 0
    else:
        new_v = 1
    surround_test.append(new_v)
test['民宿周边'] = surround_test
test.head()

,Unnamed: 0,数据ID,容纳人数,便利设施,洗手间数量,床的数量,床的类型,卧室数量,取消条款,所在城市,清洁费,首次评论日期,房主是否有个人资料图片,房主身份是否验证,房主回复率,何时成为房主,是否支持随即预订,最近评论日期,维度,经度,民宿周边,评论个数,房产类型,民宿评分,房型,邮编,便利设施_num,Internet,Kitchen,Air,Pets,Washer,aid,Dryer,Essentials,Hangers,TV,Heating,parking,24-hour
0,0,test_0,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.5,1.0,4,1.0,2,1,1,2015-05-25,t,t,100%,2015-05-20,1,2017-01-01,41.849684,-87.676270,1,17,17,97.0,1,60608,19,True,True,True,False,True,True,True,True,False,True,True,False,False
1,1,test_1,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",2.0,1.0,4,1.0,2,3,1,2015-11-09,t,t,100%,2015-09-08,0,2015-11-15,34.068613,-118.246455,1,2,0,100.0,0,90012,12,True,True,True,False,True,False,True,True,False,True,True,False,False
2,2,test_2,5,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",1.0,3.0,4,2.0,1,4,1,2017-05-15,t,t,100%,2017-05-06,1,2017-09-25,40.701958,-73.917352,1,25,0,88.0,0,11237,25,True,True,True,True,False,True,False,True,True,True,True,False,False
3,3,test_3,6,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",1.0,3.0,4,1.0,2,4,1,2012-11-12,t,t,70%,2009-02-06,0,2017-07-29,40.742959,-73.990820,1,12,0,82.0,0,10010,9,True,True,True,False,False,False,False,False,False,True,True,False,False
4,4,test_4,2,"{Internet,""Wireless Internet"",""Air conditionin...",1.0,1.0,4,1.0,0,3,1,2017-02-17,t,t,100%,2015-10-20,0,2017-03-25,34.046473,-117.734095,0,2,17,100.0,1,91766,17,True,False,True,False,False,False,False,True,True,False,True,True,False


In [9]:
test.loc[1]

Unnamed: 0                                                     1
数据ID                                                      test_1
容纳人数                                                           2
便利设施           {TV,Internet,"Wireless Internet","Air conditio...
洗手间数量                                                          2
床的数量                                                           1
床的类型                                                           4
卧室数量                                                           1
取消条款                                                           2
所在城市                                                           3
清洁费                                                            1
首次评论日期                                                2015-11-09
房主是否有个人资料图片                                                    t
房主身份是否验证                                                       t
房主回复率                                                       100%
何时成为房主                   

In [10]:
train['房主是否有个人资料图片'].isnull().sum()

160

In [11]:
# 预处理4
# 将str_cols中的每个列名对应的列进行编码，编码结果为0/1
train['房主是否有个人资料图片'] = train['房主是否有个人资料图片'].fillna('f')
test['房主是否有个人资料图片'] = test['房主是否有个人资料图片'].fillna('f')
train['房主身份是否验证'] = train['房主身份是否验证'].fillna('f')
test['房主身份是否验证'] = test['房主身份是否验证'].fillna('f')
str_cols = ['Internet','Kitchen','房主是否有个人资料图片','房主身份是否验证','Air','Pets','Washer','aid','Dryer','Essentials','Hangers','TV','Heating','parking','24-hour']
le = LabelEncoder()
for col in str_cols:
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])
train.head(10)

,Unnamed: 0,数据ID,容纳人数,便利设施,洗手间数量,床的数量,床的类型,卧室数量,取消条款,所在城市,清洁费,首次评论日期,房主是否有个人资料图片,房主身份是否验证,房主回复率,何时成为房主,是否支持随即预订,最近评论日期,维度,经度,民宿周边,评论个数,房产类型,民宿评分,房型,邮编,价格,便利设施_num,Internet,Kitchen,Air,Pets,Washer,aid,Dryer,Essentials,Hangers,TV,Heating,parking,24-hour
0,0,train_0,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.5,3.0,4,2.0,0,3,0,2015-05-07,1,1,NaN,2015-02-25,0,2016-06-26,34.109039,-118.273390,1,12,17,97.0,0,90027,64.918531,17,1,1,1,1,1,0,1,1,0,1,1,0,0
1,1,train_1,2,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",1.0,1.0,4,1.0,2,4,1,2016-07-02,1,1,NaN,2009-10-27,1,2016-07-31,40.812897,-73.919163,1,6,0,87.0,0,10454,54.918531,18,1,1,0,1,1,1,1,1,1,1,1,1,0
2,2,train_2,4,"{TV,""Air conditioning"",Kitchen,Heating,""Smoke ...",1.0,2.0,4,0.0,2,4,1,2017-07-01,1,0,100%,2017-06-29,1,2017-07-31,40.737643,-73.953309,1,4,0,80.0,0,11222,73.219281,11,0,1,1,0,0,1,0,1,0,1,1,0,0
3,3,train_3,2,{},1.0,1.0,4,1.0,0,5,1,NaN,1,1,NaN,2013-03-19,0,NaN,37.759935,-122.420558,1,0,0,NaN,1,94110,64.093909,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,train_4,3,"{Internet,""Wireless Internet"",""Air conditionin...",1.0,1.0,4,1.0,1,4,1,2014-04-30,1,1,100%,2011-07-30,0,2016-05-22,40.683363,-73.949490,1,16,0,99.0,0,11216,68.454901,13,1,1,1,1,0,0,0,1,0,0,1,1,0
5,5,train_5,5,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1.0,2.0,4,3.0,0,4,1,2016-11-08,1,1,NaN,2014-08-25,1,2017-07-02,40.714336,-73.960847,1,4,0,100.0,0,11211.0,79.657843,12,1,1,1,0,0,0,0,1,1,1,1,0,0
6,6,train_6,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,1.0,4,1.0,1,4,0,2014-02-10,1,1,89%,2014-01-01,0,2017-09-27,40.756672,-73.964642,1,113,0,96.0,1,10022,65.999128,17,1,1,1,0,0,1,0,1,1,1,1,0,1
7,7,train_7,1,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,1.0,4,1.0,0,4,0,2016-05-31,1,0,NaN,2016-03-16,0,2016-10-23,40.772482,-73.981084,1,3,0,100.0,0,10023,76.073303,20,1,1,1,0,0,0,0,1,1,1,1,0,1
8,8,train_8,3,"{""Wireless Internet"",""Air conditioning"",""Wheel...",1.0,1.0,4,1.0,1,0,1,2017-02-16,1,1,90%,2016-05-12,1,2017-10-01,42.376112,-71.050181,1,32,11,92.0,0,02129,70.000000,17,1,1,1,1,1,0,1,1,1,0,0,1,1
9,9,train_9,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,1.0,4,1.0,1,4,1,2016-10-02,1,1,100%,2012-11-05,1,2017-09-24,40.712308,-73.992568,1,51,0,95.0,0,10002,71.292830,20,1,1,1,0,0,0,0,1,1,1,1,0,1


In [12]:
# 预处理5
train['房主回复率'] = train['房主回复率'].fillna('20%')
test['房主回复率'] = test['房主回复率'].fillna('20%')
train['房主回复率'] = train['房主回复率'].str[:-1]
test['房主回复率'] = test['房主回复率'].str[:-1]
train['房主回复率'].isnull().sum()

0

In [13]:
# 预处理6
train['首次评论日期'] = train['首次评论日期'].fillna('2019-1-1')
test['首次评论日期'] = test['首次评论日期'].fillna('2019-1-1')
train['最近评论日期'] = train['最近评论日期'].fillna('2019-1-1')
test['最近评论日期'] = test['最近评论日期'].fillna('2019-1-1')
time_cols = ['首次评论日期','最近评论日期','何时成为房主']
for col in time_cols:
    train[col] = pd.to_datetime(train[col])
    test[col] = pd.to_datetime(test[col])
train['最近评论日期'][:10]

0   2016-06-26
1   2016-07-31
2   2017-07-31
3   2019-01-01
4   2016-05-22
5   2017-07-02
6   2017-09-27
7   2016-10-23
8   2017-10-01
9   2017-09-24
Name: 最近评论日期, dtype: datetime64[ns]

In [14]:
# 临时查看各列的数据类型
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59288 entries, 0 to 59287
Data columns (total 41 columns):
Unnamed: 0     59288 non-null int64
数据ID           59288 non-null object
容纳人数           59288 non-null int64
便利设施           59288 non-null object
洗手间数量          59128 non-null float64
床的数量           59184 non-null float64
床的类型           59288 non-null int64
卧室数量           59210 non-null float64
取消条款           59288 non-null int64
所在城市           59288 non-null int64
清洁费            59288 non-null int64
首次评论日期         59288 non-null datetime64[ns]
房主是否有个人资料图片    59288 non-null int64
房主身份是否验证       59288 non-null int64
房主回复率          59288 non-null object
何时成为房主         59128 non-null datetime64[ns]
是否支持随即预订       59288 non-null int64
最近评论日期         59288 non-null datetime64[ns]
维度             59288 non-null float64
经度             59288 non-null float64
民宿周边           59288 non-null int64
评论个数           59288 non-null int64
房产类型           59288 non-null int64
民宿评分           45893 no

In [15]:
train['邮编'][:10]

0      90027
1      10454
2      11222
3      94110
4      11216
5    11211.0
6      10022
7      10023
8      02129
9      10002
Name: 邮编, dtype: object

In [16]:
# 预处理7
train['邮编'] = train['邮编'].fillna(0)
test['邮编'] = test['邮编'].fillna(0)
train['邮编'] = train['邮编'].astype('str')
train['邮编'] = train['邮编'].astype('str')
train['邮编'][:10]

0      90027
1      10454
2      11222
3      94110
4      11216
5    11211.0
6      10022
7      10023
8      02129
9      10002
Name: 邮编, dtype: object

In [17]:
train.columns

Index(['Unnamed: 0', '数据ID', '容纳人数', '便利设施', '洗手间数量', '床的数量', '床的类型', '卧室数量',
       '取消条款', '所在城市', '清洁费', '首次评论日期', '房主是否有个人资料图片', '房主身份是否验证', '房主回复率',
       '何时成为房主', '是否支持随即预订', '最近评论日期', '维度', '经度', '民宿周边', '评论个数', '房产类型',
       '民宿评分', '房型', '邮编', '价格', '便利设施_num', 'Internet', 'Kitchen', 'Air',
       'Pets', 'Washer', 'aid', 'Dryer', 'Essentials', 'Hangers', 'TV',
       'Heating', 'parking', '24-hour'],
      dtype='object')

In [18]:
# 预处理8 将数据集分为连续变量和分类变量，分别进行处理
cols = train.columns
cont_cols = ['容纳人数','便利设施_num','洗手间数量',
'床的数量','卧室数量','维度', '经度','评论个数','民宿评分','房主回复率'] # 
cate_cols = ['Internet', 'Kitchen', 'Air',
       'Pets', 'Washer', 'aid', 'Dryer', 'Essentials', 'Hangers', 'TV',
       'Heating', 'parking', '24-hour','床的类型','民宿周边','取消条款',
       '所在城市', '清洁费','是否支持随即预订','房型','房主是否有个人资料图片', '房主身份是否验证','房产类型']
features = cont_cols + cate_cols
train[cate_cols] =train[cate_cols].astype('category') # 分类变量类型转换为'category'
test[cate_cols] =test[cate_cols].astype('category')

# 连续变量和分类变量分别进行不同的处理
for col in cate_cols:
       train[col] =train[col].fillna(train[col].mode()) # 对于分类变量的缺失值使用该列的众数
       test[col] =test[col].fillna(test[col].mode())

for col in ['洗手间数量','床的数量','卧室数量','民宿评分']:
       train[col] =train[col].fillna(train[col].median()) # 对于部分尚未进行处理的连续变量的缺失值使用该列的中位数
       test[col] =test[col].fillna(test[col].median())

train = train.dropna().reset_index()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59128 entries, 0 to 59127
Data columns (total 42 columns):
index          59128 non-null int64
Unnamed: 0     59128 non-null int64
数据ID           59128 non-null object
容纳人数           59128 non-null int64
便利设施           59128 non-null object
洗手间数量          59128 non-null float64
床的数量           59128 non-null float64
床的类型           59128 non-null category
卧室数量           59128 non-null float64
取消条款           59128 non-null category
所在城市           59128 non-null category
清洁费            59128 non-null category
首次评论日期         59128 non-null datetime64[ns]
房主是否有个人资料图片    59128 non-null category
房主身份是否验证       59128 non-null category
房主回复率          59128 non-null object
何时成为房主         59128 non-null datetime64[ns]
是否支持随即预订       59128 non-null category
最近评论日期         59128 non-null datetime64[ns]
维度             59128 non-null float64
经度             59128 non-null float64
民宿周边           59128 non-null category
评论个数           59128 non-null int64

In [19]:
X_train = train[features]
# 三种特征缩放的缩放器选择
# scaler = StandardScaler()
# scaler = MinMaxScaler()
scaler = RobustScaler()
scaler.fit(X_train[cont_cols]) # 对训练集中的数据进行拟合（计算数字特征）
X_train[cont_cols] = scaler.transform(X_train[cont_cols])
X_test = test[features]
X_test[cont_cols] = scaler.transform(X_test[cont_cols])# 使用拟合的缩放器对数据集中的连续变量进行特征缩放(scaler.transform)
y_train = train['价格'].astype('int')

In [20]:
X_train.head()

,容纳人数,便利设施_num,洗手间数量,床的数量,卧室数量,维度,经度,评论个数,民宿评分,房主回复率,Internet,Kitchen,Air,Pets,Washer,aid,Dryer,Essentials,Hangers,TV,Heating,parking,24-hour,床的类型,民宿周边,取消条款,所在城市,清洁费,是否支持随即预订,房型,房主是否有个人资料图片,房主身份是否验证,房产类型
0,1.0,0.000000,0.5,2.0,1.0,-0.990236,-0.929903,0.272727,0.166667,-1.0,1,1,1,0,1,0,1,1,1,1,1,0,0,4,1,2,4,1,0,0,1,1,0
1,0.0,0.111111,0.0,0.0,0.0,0.022711,0.069343,0.000000,-1.500000,-1.0,1,1,0,1,1,1,1,1,1,1,1,1,0,4,1,2,4,1,1,0,1,1,0
2,1.0,-0.666667,0.0,1.0,-1.0,0.011341,0.068573,-0.090909,-2.666667,0.0,0,1,1,0,0,1,0,1,0,1,1,0,0,4,1,2,4,1,1,0,1,0,0
3,0.0,-1.777778,0.0,0.0,0.0,-0.438589,-1.023334,-0.272727,0.000000,-1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0,5,1,0,1,1,1,0
4,0.5,-0.444444,0.0,0.0,0.0,0.003139,0.068659,0.454545,0.500000,0.0,1,1,1,1,0,0,0,1,0,0,1,1,0,4,1,1,4,1,0,0,1,1,0


In [21]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14823 entries, 0 to 14822
Data columns (total 33 columns):
容纳人数           14823 non-null float64
便利设施_num       14823 non-null float64
洗手间数量          14823 non-null float64
床的数量           14823 non-null float64
卧室数量           14823 non-null float64
维度             14823 non-null float64
经度             14823 non-null float64
评论个数           14823 non-null float64
民宿评分           14823 non-null float64
房主回复率          14823 non-null float64
Internet       14823 non-null category
Kitchen        14823 non-null category
Air            14823 non-null category
Pets           14823 non-null category
Washer         14823 non-null category
aid            14823 non-null category
Dryer          14823 non-null category
Essentials     14823 non-null category
Hangers        14823 non-null category
TV             14823 non-null category
Heating        14823 non-null category
parking        14823 non-null category
24-hour        14823 non-null category
床的类型

### 3.划分数据集，构建回归模型

In [22]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,random_state = 888,test_size = 0.2)

In [23]:
from catboost import CatBoostRegressor,Pool

In [24]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47302 entries, 19270 to 10750
Data columns (total 33 columns):
容纳人数           47302 non-null float64
便利设施_num       47302 non-null float64
洗手间数量          47302 non-null float64
床的数量           47302 non-null float64
卧室数量           47302 non-null float64
维度             47302 non-null float64
经度             47302 non-null float64
评论个数           47302 non-null float64
民宿评分           47302 non-null float64
房主回复率          47302 non-null float64
Internet       47302 non-null category
Kitchen        47302 non-null category
Air            47302 non-null category
Pets           47302 non-null category
Washer         47302 non-null category
aid            47302 non-null category
Dryer          47302 non-null category
Essentials     47302 non-null category
Hangers        47302 non-null category
TV             47302 non-null category
Heating        47302 non-null category
parking        47302 non-null category
24-hour        47302 non-null category


In [25]:
y_train.index

Int64Index([19270, 24438, 22451, 54250, 48458, 52778,  2168, 23645,  7681,
             9789,
            ...
            43708, 57337, 49966, 29031, 56435,  5270, 58607, 14829, 34714,
            10750],
           dtype='int64', length=47302)

In [26]:
train_pool = Pool(X_train,y_train)
eval_pool = Pool(X_valid,y_valid)
#创建CatBoost回归模型
model = CatBoostRegressor()
model.fit(train_pool,eval_set = eval_pool)

0:	learn: 67.2758620	test: 67.2890103	best: 67.2890103 (0)	total: 81.5ms	remaining: 1m 21s
1:	learn: 65.2902804	test: 65.3047044	best: 65.3047044 (1)	total: 171ms	remaining: 1m 25s
2:	learn: 63.3614681	test: 63.3750122	best: 63.3750122 (2)	total: 272ms	remaining: 1m 30s
3:	learn: 61.4987887	test: 61.5118621	best: 61.5118621 (3)	total: 372ms	remaining: 1m 32s
4:	learn: 59.6882216	test: 59.7001348	best: 59.7001348 (4)	total: 471ms	remaining: 1m 33s
5:	learn: 57.9340313	test: 57.9460272	best: 57.9460272 (5)	total: 571ms	remaining: 1m 34s
6:	learn: 56.2477455	test: 56.2600904	best: 56.2600904 (6)	total: 669ms	remaining: 1m 34s
7:	learn: 54.5968523	test: 54.6090972	best: 54.6090972 (7)	total: 679ms	remaining: 1m 24s
8:	learn: 53.0108590	test: 53.0223077	best: 53.0223077 (8)	total: 778ms	remaining: 1m 25s
9:	learn: 51.4586972	test: 51.4705827	best: 51.4705827 (9)	total: 878ms	remaining: 1m 26s
10:	learn: 49.9627200	test: 49.9763715	best: 49.9763715 (10)	total: 975ms	remaining: 1m 27s
11:	lea

In [27]:
y_predict_valid = model.predict(X_valid)
y_predict_valid

array([57.10898753, 70.11006892, 57.27671103, ..., 68.47169452,
       61.95818395, 66.86794883])

In [28]:
# 合并
result_df = X_valid.copy()
result_df['价格'] = y_valid
result_df['预测价格'] = y_predict_valid
result_df.head()

,容纳人数,便利设施_num,洗手间数量,床的数量,卧室数量,维度,经度,评论个数,民宿评分,房主回复率,Internet,Kitchen,Air,Pets,Washer,aid,Dryer,Essentials,Hangers,TV,Heating,parking,24-hour,床的类型,民宿周边,取消条款,所在城市,清洁费,是否支持随即预订,房型,房主是否有个人资料图片,房主身份是否验证,房产类型,价格,预测价格
46460,0.0,-0.222222,1.0,1.0,0.0,0.003064,0.070384,0.863636,-1.000000,0.0,1,1,0,0,0,1,0,1,1,1,0,0,0,4,1,2,4,1,0,1,1,1,0,57,57.108988
13310,1.5,-0.555556,0.0,0.0,3.0,0.203770,-0.240326,1.045455,-1.333333,0.0,1,1,1,0,1,0,0,0,0,1,1,0,0,4,1,2,1,1,0,0,1,1,0,68,70.110069
5389,-0.5,-0.444444,0.0,0.0,0.0,0.003541,0.068702,-0.181818,0.666667,-1.0,1,1,0,1,0,0,0,1,1,1,1,0,0,4,1,2,4,0,1,1,1,0,0,50,57.276711
25921,0.5,-0.222222,0.0,0.0,-1.0,-0.991054,-0.931253,-0.272727,0.000000,-1.0,1,1,1,0,0,0,0,1,0,1,1,1,0,4,1,0,3,1,0,0,1,0,0,70,69.416098
53807,0.0,1.111111,0.0,0.0,0.0,0.024785,0.069382,-0.181818,0.666667,0.0,1,1,1,0,0,0,0,1,0,1,1,0,0,4,1,0,4,0,1,1,1,0,0,61,59.911521


In [31]:
# 导入评价指标
from sklearn.metrics import mean_squared_error,r2_score
#计算均方误差
mse = mean_squared_error(y_valid,y_predict_valid)
r2 = r2_score(y_valid,y_predict_valid)
print(f"mse:{mse}  r2:{r2}")

mse:32.4909731998325  r2:0.6944082630941844


In [32]:
prediction  = model.predict(X_test)
# 合并
new_result_df = X_test.copy()
new_result_df['预测价格'] = prediction
new_result_df.head()

,容纳人数,便利设施_num,洗手间数量,床的数量,卧室数量,维度,经度,评论个数,民宿评分,房主回复率,Internet,Kitchen,Air,Pets,Washer,aid,Dryer,Essentials,Hangers,TV,Heating,parking,24-hour,床的类型,民宿周边,取消条款,所在城市,清洁费,是否支持随即预订,房型,房主是否有个人资料图片,房主身份是否验证,房产类型,预测价格
0,0.0,0.222222,0.5,0.0,0.0,0.179369,-0.240588,0.500000,0.166667,0.000,1,1,1,0,1,0,1,1,1,1,1,0,0,4,1,2,4,1,0,0,1,1,0,70.724567
1,0.0,-0.555556,1.0,0.0,0.0,-0.996344,-0.929297,-0.181818,0.666667,0.000,1,1,1,0,1,0,1,1,0,1,1,0,0,4,1,2,3,1,0,0,1,1,0,69.358871
2,1.5,0.888889,0.0,2.0,1.0,0.005949,0.069383,0.863636,-1.333333,0.000,1,1,1,1,0,1,0,1,1,1,1,0,0,4,1,1,4,1,1,0,1,1,0,69.485009
3,2.0,-0.888889,0.0,2.0,0.0,0.012144,0.067728,0.272727,-2.333333,-0.375,1,1,1,0,0,0,0,0,0,1,1,0,0,4,1,2,4,1,0,0,1,1,0,77.645018
4,0.0,0.000000,0.0,0.0,0.0,-0.999690,-0.917754,-0.181818,0.666667,0.000,1,0,1,0,0,0,0,1,1,0,1,1,0,4,0,0,3,1,0,1,1,1,17,56.139437
